In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import cvxopt

In [21]:
# !pip install cvxopt

![imgs/tarea_kernels1.png](imgs/tarea_kernels1.png)

In [10]:
data = pd.DataFrame({"x1": [2,2,-2,-2,1,-1,1,-1], 
                     "x2": [2,-2,-2,2,1,-1,-1,1],
                     "clase": [1,1,1,1,-1,-1,-1,-1]    
                     })
data["size"] = 10
fig = px.scatter(data, x='x1', y='x2', color='clase', size="size")
fig.show()

# plt.scatter(data.x1, data.x2, c=data.clase, s=80)
# plt.legend()

In [13]:
data_t1 = data
data_t1["new_x1"] = data_t1["x1"]**2
data_t1["new_x2"] = np.sqrt(2) * data_t1["x1"] * data_t1["x2"] 
data_t1["new_x3"] = data_t1["x2"]**2
display(data_t1)
fig = px.scatter_3d(data_t1, x='new_x1', y='new_x2', z='new_x3',
              color='clase')
fig.show()

,x1,x2,clase,size,new_x1,new_x2,new_x3
0,2,2,1,10,4,5.656854,4
1,2,-2,1,10,4,-5.656854,4
2,-2,-2,1,10,4,5.656854,4
3,-2,2,1,10,4,-5.656854,4
4,1,1,-1,10,1,1.414214,1
5,-1,-1,-1,10,1,1.414214,1
6,1,-1,-1,10,1,-1.414214,1
7,-1,1,-1,10,1,-1.414214,1


In [14]:
data_t2 = data
data_t2["new_x1"] = data_t1["x1"]**2
data_t2["new_x2"] = data_t1["x2"]**2
data_t2["new_x3"] = (data_t1["x1"]**2 + data_t1["x2"]**2 -  5) /3
display(data_t1)
fig = px.scatter_3d(data_t2, x='new_x1', y='new_x2', z='new_x3',
              color='clase')
fig.show()

,x1,x2,clase,size,new_x1,new_x2,new_x3
0,2,2,1,10,4,4,1.0
1,2,-2,1,10,4,4,1.0
2,-2,-2,1,10,4,4,1.0
3,-2,2,1,10,4,4,1.0
4,1,1,-1,10,1,1,-1.0
5,-1,-1,-1,10,1,1,-1.0
6,1,-1,-1,10,1,1,-1.0
7,-1,1,-1,10,1,1,-1.0


![imgs/tarea_kernels1.png](imgs/tarea_kernels2.png)

In [27]:
X = data_t1[["new_x1","new_x2","new_x3"]].values
y = data_t1[["clase"]].values
print(f'X: {X.shape}, Y: {y.shape}')


X: (8, 3), Y: (8, 1)


In [32]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [39]:
#Initializing values and computing H. Note the 1. to force to float type
m,n = X.shape
y = y.reshape(-1,1) * 1.
X_dash = y * X
H = np.dot(X_dash , X_dash.T) * 1.

#Converting into cvxopt format
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(-np.eye(m))
h = cvxopt_matrix(np.zeros(m))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Setting solver parameters (change default to decrease tolerance) 
cvxopt_solvers.options['show_progress'] = True
cvxopt_solvers.options['abstol'] = 1e-10
cvxopt_solvers.options['reltol'] = 1e-10
cvxopt_solvers.options['feastol'] = 1e-10

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

#w parameter in vectorized form
w = ((y * alphas).T @ X).reshape(-1,1)

#Selecting the set of indices S corresponding to non zero parameters
S = (alphas > 1e-4).flatten()

#Computing b
b = y[S] - np.dot(X[S], w)

#Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b[0])

     pcost       dcost       gap    pres   dres
 0: -9.0864e-02 -2.6864e-01  2e-01  2e-17  1e+00
 1: -9.0906e-02 -9.2713e-02  2e-03  9e-18  2e-02
 2: -9.0909e-02 -9.0927e-02  2e-05  8e-18  2e-04
 3: -9.0909e-02 -9.0909e-02  2e-07  1e-17  2e-06
 4: -9.0909e-02 -9.0909e-02  2e-09  1e-17  2e-08
 5: -9.0909e-02 -9.0909e-02  2e-11  1e-17  2e-10
 6: -9.0909e-02 -9.0909e-02  2e-13  2e-17  2e-12
Optimal solution found.
Alphas =  [0.02272727 0.02272727 0.02272727 0.02272727 0.02272727 0.02272727
 0.02272727 0.02272727]
w =  [0.27272727 0.27272727 0.18181818]
b =  [-1.36363636]


# References:

* [1] [https://plotly.com/python/3d-scatter-plots/](https://plotly.com/python/3d-scatter-plots/)
* [2] [https://www.robots.ox.ac.uk/~az/lectures/ml/lect3.pdf](https://www.robots.ox.ac.uk/~az/lectures/ml/lect3.pdf)
* [3] [https://xavierbourretsicotte.github.io/SVM_implementation.html](https://xavierbourretsicotte.github.io/SVM_implementation.html)